In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
file_train = '/kaggle/input/tabular-playground-series-jul-2021/train.csv'
file_test = '/kaggle/input/tabular-playground-series-jul-2021/test.csv'

In [ ]:
home_data_train = pd.read_csv(file_train)
home_data_test = pd.read_csv(file_test)

In [ ]:
home_data_train.head()

In [ ]:
home_data_test.head()

In [ ]:
y1 = home_data_train.target_carbon_monoxide
y2 = home_data_train.target_benzene
y3 = home_data_train.target_nitrogen_oxides

In [ ]:
home_data_train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides', 'date_time'], axis=1, inplace=True)
home_data_test.drop(['date_time'], axis=1, inplace=True)
X_train = home_data_train
X_test = home_data_test

In [ ]:
cols_w_m_n_train = [col for col in X_train.columns if X_train[col].isnull().any()]
cols_w_m_n_test = [col for col in X_test.columns if X_test[col].isnull().any()]
print(cols_w_m_n_train, '\n',cols_w_m_n_test)

In [ ]:
s = (X_train.dtypes == 'object')
cols_s = list(s[s].index)

ss = (X_test.dtypes == 'object')
cols_ss = list(s[s].index)

print("Categorical variables in train data:", cols_s)
print("Categorical variables in test data:", cols_ss)

In [ ]:
n_unique_train = list(map(lambda col: X_train[col].nunique(), cols_s))
d = dict(zip(cols_s, n_unique_train))
sorted(d.items(), key = lambda x: x[1])

In [ ]:
n_unique_test = list(map(lambda col: X_test[col].nunique(), cols_ss))
dd = dict(zip(cols_ss, n_unique_test))
sorted(dd.items(), key = lambda x: x[1])

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

px.imshow(X_train.corr(),
         height = 600, width = 600)

In [ ]:
X_train.drop(['sensor_3'], axis=1, inplace=True)
X_test.drop(['sensor_3'], axis=1, inplace=True)

In [ ]:
X_test.head()

In [ ]:
px.imshow(X_train.corr(),
         height = 600, width = 600)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
from xgboost import XGBRegressor
s_X_train, s_X_test, s_y_train, s_y_test = train_test_split(X_train, y1, train_size=0.80, test_size=0.20, random_state=0)
model = XGBRegressor(n_estimators = 10000, learning_rate = 0.095)
model.fit(s_X_train, s_y_train, early_stopping_rounds = 55, eval_set = [(s_X_test, s_y_test)], verbose = False)
val_predictions = model.predict(s_X_test)
val_mae = mean_absolute_error(val_predictions, s_y_test)
print(val_mae)

In [ ]:
s_X_train, s_X_test, s_y_train, s_y_test = train_test_split(X_train, y2, train_size=0.80, test_size=0.20, random_state=1)
model.fit(s_X_train, s_y_train, early_stopping_rounds = 55, eval_set = [(s_X_test, s_y_test)], verbose = False)
print(val_mae)

In [ ]:
s_X_train, s_X_test, s_y_train, s_y_test = train_test_split(X_train, y3, train_size=0.80, test_size=0.20, random_state=1)
model.fit(s_X_train, s_y_train, early_stopping_rounds = 55, eval_set = [(s_X_test, s_y_test)], verbose = False)
print(val_mae)

In [ ]:
print(y1)

In [ ]:
X_test.deg_C

In [ ]:
X_train.deg_C

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
model.fit(X_train, y1, early_stopping_rounds = 55, eval_set = [(X_train, y1)], verbose = False)
carbon_monoxide = model.predict(X_test)

model.fit(X_train, y2, early_stopping_rounds = 55, eval_set = [(X_train, y2)], verbose = False)
benzene = model.predict(X_test)

model.fit(X_train, y3, early_stopping_rounds = 55, eval_set = [(X_train, y3)], verbose = False)
nitrogen_oxides = model.predict(X_test)

output = pd.DataFrame({'date_time': pd.read_csv(file_test).date_time,
                       'target_carbon_monoxide': carbon_monoxide,
                       'target_benzene': benzene,
                       'target_nitrogen_oxides': nitrogen_oxides})

In [ ]:
output.head()

In [ ]:
output.to_csv('submission3.csv', index=False)